### Import libraries

In [1]:
print("Starting")

Starting


In [2]:
import awkward as ak
from consistent_plots import hist, hist2d
import matplotlib as mpl
mpl.style.use('../../presentation.mplstyle')
mpl.rcParams['font.size'] = 14
import matplotlib.pyplot as plt
%matplotlib widget
import numpy as np
from trsm import TRSM, combos
import vector

In [11]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
# mpl.rcParams.keys()

In [20]:
def norm_hist(arr, bins=100):
    n, b = np.histogram(arr, bins=bins)
    x = (b[:-1] + b[1:]) / 2
    
    return n/n.max(), b, x

### Load model

In [21]:
from keras.models import model_from_json

In [22]:
from pickle import load

In [23]:
tag = 'lr_0pt0001_beta1_0pt1_beta2_0pt5'

In [24]:
json_file = open(f'../models/{tag}/model/model_1.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
loaded_model.load_weights(f'../models/{tag}/model/model_1.h5')

In [25]:
scaler = load(open(f'../models/{tag}/model/scaler_1.pkl', 'rb'))

### Open ROOT file

In [3]:
filename = f'../../signal/skimmed/NMSSM_XYH_YToHH_6b_MX_700_MY_400_testing_set_skimmed.root'
trsm = TRSM(filename=filename)

-- [INFO] -- /eos/user/s/srosenzw/miniconda3/envs/work/lib/python3.8/runpy.py -- Opening ROOT file ../../signal/skimmed/NMSSM_XYH_YToHH_6b_MX_700_MY_400_testing_set_skimmed.root with columns
-- [INFO] -- /eos/user/s/srosenzw/miniconda3/envs/work/lib/python3.8/runpy.py -- Tree contains 52891 events.
----------------------------------------------------------------------------------------------------
                                            TABLE COLUMNS                                           
----------------------------------------------------------------------------------------------------
jet_pt                            jet_eta                           jet_phi                           
jet_m                             jet_btag                          jet_qgl                           
jet_idx                           jet_hadronFlav                    jet_partonFlav                    
HX_b1_recojet_m                   HX_b1_recojet_pt                  HX_b1_recojet_eta   

### Load combinations of 7 total jets

In [18]:
combos7 = combos(trsm, 7)

100%|██████████| 52891/52891 [00:33<00:00, 1582.96it/s]


Total events chosen: 8052


In [26]:
test_features_7 = scaler.transform(combos7.combo_features)
scores_7 = loaded_model.predict(test_features_7)[:,0]
scores_7.shape

(56364,)

In [27]:
sgnl_scores_7 = scores_7[combos7.sgnl_mask]
bkgd_scores_7 = scores_7[~combos7.sgnl_mask]

In [28]:
import awkward as ak

In [29]:
bkgd_parton_id = np.array((ak.flatten(combos7.bkgd_dict['partonFlav'])))
bkgd_hadron_id = np.array((ak.flatten(combos7.bkgd_dict['hadronFlav'])))

In [30]:
swap_sgnl_idx = np.array((ak.flatten(combos7.bkgd_dict['ind'])))

In [31]:
swap_sgnl_idx

array([6, 6, 6, ..., 9, 9, 9])

In [32]:
bkgd_pt = combos7.swapped_bkgd_pt
bkgd_pt = ak.to_numpy(bkgd_pt)

In [33]:
sgnl_pt = ak.to_numpy(combos7.swapped_sgnl_pt)

In [34]:
bkgd_pt = bkgd_pt.reshape(bkgd_pt.shape[0])
sgnl_pt = sgnl_pt.reshape(sgnl_pt.shape[0])

In [35]:
sgnl_p4 = combos7.incorrect_sgnl_p4
bkgd_p4 = combos7.incorrect_bkgd_p4

In [36]:
dR_incorrect = ak.to_numpy(combos7.incorrect_sgnl_p4.deltaR(combos7.incorrect_bkgd_p4))
dR_incorrect = dR_incorrect.reshape(dR_incorrect.shape[0])

In [45]:
fig, ax = plt.subplots()

n, b, x = hist(ax, dR_incorrect)
ax.set_xlabel(r'$\Delta R$ Between Swapped Jets')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0, '$\\Delta R$ Between Swapped Jets')

In [44]:
fig, ax = plt.subplots()

n, x, y, im = hist2d(ax, dR_incorrect, scores_7[~combos7.sgnl_mask], xbins=np.linspace(0,6,100), ybins=np.linspace(0,1.01,100))
ax.set_xlim(0,5.5)
plt.colorbar(im, ax=ax)

ax.set_xlabel(r"$\Delta R$ Between Swapped Jets")
ax.set_ylabel("Assigned Score")

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [46]:
fig, ax = plt.subplots()

n, xedges, yedges, im = hist2d(ax, sgnl_pt, bkgd_pt, xbins=np.arange(30,200), ybins=np.arange(30,200))
plt.colorbar(im, ax=ax)
ax.set_xlabel(r"Signal jet $p_T$ [GeV]")
ax.set_ylabel(r"Swapped jet $p_T$ [GeV]")

plt.tight_layout()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [60]:
fig, ax = plt.subplots()

sc = ax.scatter(sgnl_pt, bkgd_pt, c=bkgd_scores_7, cmap='rainbow', alpha=0.75, s=1, vmin=0.0, vmax=1.0)
plt.colorbar(sc)
ax.set_xlabel(r"Signal jet $p_T$ [GeV]")
ax.set_xlim(0,500)
ax.set_ylabel(r"Swapped jet $p_T$ [GeV]")
ax.set_ylim(0,800)

plt.tight_layout()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [57]:
fig, ax = plt.subplots()

high_mask = bkgd_scores_7 > 0.8
low_mask = bkgd_scores_7 <= 0.8

sc = ax.scatter(sgnl_pt[low_mask], bkgd_pt[low_mask], alpha=0.75, s=1, label=r'score $\leq$ 0.8')
sc = ax.scatter(sgnl_pt[high_mask], bkgd_pt[high_mask], alpha=0.75, s=1, label=r'score $>$ 0.8')
ax.set_xlabel(r"Signal jet $p_T$ [GeV]")
ax.set_xlim(0,500)
ax.set_ylabel(r"Swapped jet $p_T$ [GeV]")
ax.set_ylim(0,800)
ax.legend()

plt.tight_layout()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [58]:
fig, ax = plt.subplots()

n, xedges, yedges, im = hist2d(ax, scores_7[~combos7.sgnl_mask], bkgd_pt, ybins=np.arange(30,200))
plt.colorbar(im, ax=ax)
ax.set_xlabel("Assigned Score")
ax.set_ylabel(r"$p_T [GeV]$ of Incorrect Swapped Jet")

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [59]:
fig, ax = plt.subplots()

ax.set_title(r"$^7C_6$ Background pT")

n_7, b_7, x_7 = norm_hist(bkgd_pt[scores_7[~combos7.sgnl_mask] > 0.8]   , bins=np.linspace(0,600,100))
w_n_7, b_7, x_7 = norm_hist(bkgd_pt[scores_7[~combos7.sgnl_mask] <= 0.8], bins=np.linspace(0,600,100))

hist(ax, x_7, weights=n_7, bins=b_7, label='Score > 0.8')
hist(ax, x_7, weights=w_n_7, bins=b_7, label='Score <= 0.8')
ax.legend(fontsize='small', loc=9)

ax.set_xlabel('Swapped bkgd pT')
ax.set_ylabel('AU')

textstr = f'Entries = {len(scores_7)}'
props = dict(boxstyle='round', facecolor='white', alpha=1)
ax.text(0.8, 1.02, textstr, transform=ax.transAxes, fontsize=9,
        verticalalignment='top', bbox=props)

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [61]:
fig, ax = plt.subplots()

n, xedges, yedges, im = hist2d(ax, scores_7[~combos7.sgnl_mask], bkgd_parton_id, ybins=np.arange(-5,22))
plt.colorbar(im, ax=ax)
ax.set_xlabel("Assigned Score")
ax.set_ylabel("Parton ID of Incorrect Swapped Jet")

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [62]:
d_mask = bkgd_parton_id == 1
u_mask = bkgd_parton_id == 2
s_mask = bkgd_parton_id == 3
c_mask = bkgd_parton_id == 4
b_mask = bkgd_parton_id == 5

dbar_mask = bkgd_parton_id == -1
ubar_mask = bkgd_parton_id == -2
sbar_mask = bkgd_parton_id == -3
cbar_mask = bkgd_parton_id == -4
bbar_mask = bkgd_parton_id == -5

g_mask = bkgd_parton_id == 21

unknown = bkgd_parton_id == 0

In [63]:
fig, axs = plt.subplots(nrows=2, ncols=6, figsize=(15,4))

ax = axs[0][0]
hist(ax, scores_7[~combos7.sgnl_mask][d_mask])
ax.set_title('d')

ax = axs[0][1]
hist(ax, scores_7[~combos7.sgnl_mask][u_mask])
ax.set_title('u')

ax = axs[0][2]
hist(ax, scores_7[~combos7.sgnl_mask][s_mask])
ax.set_title('s')

ax = axs[0][3]
hist(ax, scores_7[~combos7.sgnl_mask][c_mask])
ax.set_title('c')

ax = axs[0][4]
hist(ax, scores_7[~combos7.sgnl_mask][b_mask])
ax.set_title('b')

ax = axs[0][5]
hist(ax, scores_7[~combos7.sgnl_mask][g_mask])
ax.set_title('g')



ax = axs[1][0]
hist(ax, scores_7[~combos7.sgnl_mask][dbar_mask])
ax.set_title('dbar')

ax = axs[1][1]
hist(ax, scores_7[~combos7.sgnl_mask][ubar_mask])
ax.set_title('ubar')

ax = axs[1][2]
hist(ax, scores_7[~combos7.sgnl_mask][sbar_mask])
ax.set_title('sbar')

ax = axs[1][3]
hist(ax, scores_7[~combos7.sgnl_mask][cbar_mask])
ax.set_title('cbar')

ax = axs[1][4]
hist(ax, scores_7[~combos7.sgnl_mask][bbar_mask])
ax.set_title('bbar')

ax = axs[1][5]
hist(ax, scores_7[~combos7.sgnl_mask][unknown])
ax.set_title('unknown')

for ax in axs.flat:
    ax.set_xlabel("Assigned Score")
    ax.set_ylabel("Count")

plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [54]:
print(swap_sgnl_idx.min(), swap_sgnl_idx.max())

0 11


In [55]:
plt.close('all')

In [56]:
fig, ax = plt.subplots()

n, x, y, im = hist2d(ax, swap_sgnl_idx, scores_7[~combos7.sgnl_mask], xbins=np.linspace(0,6,13))
ax.set_xlim(0,5.5)
plt.colorbar(im, ax=ax)

ax.set_ylabel("Assigned Score")

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …